<a href="https://colab.research.google.com/github/gregrell/CAP6671/blob/main/CAP6671_CombinedArms_Greg_Rell_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This code example uses [PettingZoo](https://www.pettingzoo.ml/#) as a mutli-agent reinforcement learning platform for the [Combinded Arms](https://www.pettingzoo.ml/magent/combined_arms) environment.

This is an example of how to render PettingZoo's GUI in Colab and also serves as a template for testing separate Red and Blue models.

Author: Dr. S. Mondesire

Last Modified: 2022-04-03

# Import Libraries

In [7]:
!apt update && apt install xvfb
!pip install gym-notebook-wrapper
!apt update && apt install python-opengl ffmpeg
!pip install pygame
!pip install pettingzoo[magent]

!pip install stable_baselines3

!pip install supersuit


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubu

# Model Creation
This model randomly moves, attacks, or does nothing per timestep. Handles both ranged and melee agents.

In [1]:
from stable_baselines3.common.vec_env.base_vec_env import VecEnvWrapper
class MyModel:
  # Imports
  from stable_baselines3.common.monitor import Monitor
  from stable_baselines3.common.vec_env import VecEnv
  

  def predict(self, env, observation, agent):
    # action = env.action_space(agent).sample()
    action = 0
    if "redmelee" in agent:
      action = 1
    elif "redranged" in agent:
      action = 2
    return action

"""
  def train(self, env):
    env = VecEnv
    """




class RandModel:
  def predict(self, env, observation, agent):
    action = env.action_space(agent).sample()
    return action

# Test Model

In [17]:
import gym
import gnwrapper
from pettingzoo.magent import combined_arms_v5
import pickle

from stable_baselines3 import PPO
from stable_baselines3.a2c import a2c

import supersuit as ss

MAX_TIMESTEPS = 500

# Initialize the Combined Arms environment
env = combined_arms_v5.parallel_env() # Parallel environemnt
env = gnwrapper.LoopAnimation(env) # Start Xvfb

num_envs = 1
env = ss.gym_vec_env_v0(env, num_envs, multiprocessing=False)
#env = ss.pettingzoo_env_to_vec_env_v1(env)



# Instantiate models
redModel = MyModel()
blueModel = RandModel()

"""
Save (pickle) model
pickle.dump(redModel, open("redModel.pkl", 'wb') )

# Load pickled model
with open("redModel.pkl", 'rb') as pickle_model:
  redModel = pickle.load(pickle_model)
"""
observations = env.reset()

model = a2c.A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000) 



for step in range(MAX_TIMESTEPS):
  actions = {}

  # For each agent, get the next action
  for agent in env.agents:
    model = None
    if "red" in agent:
      model = redModel
    elif "blue" in agent:
      model = blueModel
    
    # actions.update({agent: model.predict(env, observations[agent], agent) })

  # observations, rewards, dones, infos = env.step(actions)
  
  env.render() # Render the current frame

# Display the rendered movie
env.display()

# Final Stats
redMelee = 0
redRanged = 0
blueMelee = 0
blueRanged = 0
for agent in env.agents:
  if "redmelee" in agent:
    redMelee += 1
  elif "redranged" in agent:
    redRanged += 1
  elif "bluemele" in agent:
    blueMelee += 1
  elif "blueranged" in agent:
    blueRanged += 1

print("Red Total {}: (Melee {}, Ranged {})".format(redMelee+redRanged,redMelee, redRanged) )
print("Blue Total {}: (Melee {}, Ranged {})".format(blueMelee+blueRanged,blueMelee, blueRanged) )



TypeError: ignored

In [21]:
!pip install pettingzoo[butterfly]

     |████████████████████████████████| 980 kB 7.5 MB/s 


## IT worked???

In [30]:
from stable_baselines3 import PPO
from pettingzoo.butterfly import pistonball_v6
import supersuit as ss
#env = pistonball_v6.parallel_env()
env = combined_arms_v5.parallel_env()
#env = ss.color_reduction_v0(env, mode='B')
#env = ss.resize_v0(env, x_size=84, y_size=84)
#env = ss.frame_stack_v1(env, 3)
env = ss.black_death_v3(env)
env = ss.pad_action_space_v0(env)
env = ss.pettingzoo_env_to_vec_env_v1(env)
env = ss.concat_vec_envs_v1(env, 8, num_cpus=4, base_class='stable_baselines3')
model = PPO('MlpPolicy', env, verbose=3, n_steps=16)
#model.learn(total_timesteps=2000000)
model.learn(total_timesteps=2000)

Using cpu device


/usr/local/lib/python3.7/dist-packages/magent/gridworld.py:356: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  buf = np.empty((n,), dtype=np.bool)
/usr/local/lib/python3.7/dist-packages/magent/gridworld.py:356: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  buf = np.empty((n,), dtype=np.bool)
/usr/local/lib/python3.7/dist-packages/magent/gridworld.py

------------------------------
| time/              |       |
|    fps             | 8654  |
|    iterations      | 1     |
|    time_elapsed    | 2     |
|    total_timesteps | 20736 |
------------------------------
